Simulated Patient 

In [ ]:
import pandas as pd
import numpy as np
import os

from textwrap import wrap
from matplotlib import pyplot as plt
from stable_baselines3 import PPO, A2C, DQN
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from environment.profiles import profiles


In [ ]:
from environment.fogg_behavioural_model import Patient

In [ ]:
UpdatedEveryDayHours = 24
WeekHours = 7 * UpdatedEveryDayHours
ThreeWeeksHours = WeekHours * 3
EightWeeksHours = WeekHours * 8
FiveWeeksHours = WeekHours * 5

Condition = 'stable'
Habituation = False
#very high e.g. 9999999999 to make it outside of the intervention time...effectively no preference shift
TimePreferenceUpdateStep = 9999999999999999
# 500 runs in the paper
RunCount = 50

In [ ]:
ResultsDirectory = 'results'
if not os.path.exists(ResultsDirectory):
  os.makedirs(ResultsDirectory)

# Plots

In [ ]:
from typing import NamedTuple

class ExperimentResult(NamedTuple):
  sleep_hours: list[float]
  positive_mood_hours: list[float]
  response_ratios: list[float]
  notifications: list[float]


In [ ]:
def plot_experiment_result_hours(result: ExperimentResult):
  (sleeps_hours, positive_moods_hours, _, _) = result

  plt.figure(figsize=(8, 5))
  plt.plot(np.nanmean(sleeps_hours, axis=0), label='Hours slept', color='r')
  plt.plot(np.mean(positive_moods_hours, axis=0), label='Hours in positive mood')
  plt.ylabel('Hours ')
  plt.xlabel('Intervention Days')
  plt.legend()
  plt.show()

def plot_experiment_result_ratios(result: ExperimentResult):
  (_, _, ratios, notifications) = result

  _, (ratios_axis, notifications_axis) = plt.subplots(2, 1, sharex=True, sharey=False, figsize=(10, 8))
  ratios_axis.plot(np.nanmean(ratios, axis=0), label='response ratio', color='r')
  notifications_axis.plot(np.nanmean(notifications, axis=0), label='num of notifications')
  notifications_axis.set_xlabel('Intervention days')
  notifications_axis.set_ylabel('No. of notifications')
  ratios_axis.set_ylabel('Response ratio')
  plt.show()

class ReadResult(NamedTuple):
  ratios: list[float]
  notifications: list[float]

def read_ratios_and_notifications(name: str, epoch_count: int = RunCount, condition: str = Condition) -> ReadResult:
  ratios = []
  notifications = []

  no_activity_count = 0
  for epoch in range(epoch_count):
    frame = pd.read_csv(f"{ResultsDirectory}/patient1_{condition}_{name}_run{epoch}.csv")
    has_performed_activity = any(frame['activity performed'].values)

    if has_performed_activity:
      ratios.append(frame.response_ratio)
      notifications.append(frame.notifications)
    else:
      no_activity_count += 1

  print(f" {no_activity_count} out of {epoch_count} runs had no activity performed throughout the full intervention.")
  return ReadResult(ratios, notifications)

def plot_experiment_results_ratio(label: str, not_random_result: ReadResult, result_map: dict[str, ReadResult]):
  figure: plt.Figure
  figure, (ratios_axis, notifications_axis) = plt.subplots(2, 1, sharex=True, sharey=False, figsize=(10, 10))
  figure.suptitle(f'profile "{label}": Comparison of prompt learning strategies', fontsize=20)
  figure.subplots_adjust(top=0.9)

  for label, (ratios, notifications) in result_map.items():
    ratios_axis.plot(np.nanmean(ratios, axis=0), label=label)
    notifications_axis.plot(np.nanmean(notifications, axis=0), label=label)

  notifications_axis.plot(
    np.ones(len(np.mean(not_random_result.notifications, axis=0))) + 2,
    label='Preferred number of notifications a day'
  )
  notifications_axis.set_xlabel('Intervention Days', fontsize=16)
  notifications_axis.set_ylabel('\n'.join(wrap('Numbers of notifications', 20)), fontsize=16)
  ratios_axis.set_ylabel('\n'.join(wrap('Activity performed to prompt ratio', 20)))
  notifications_axis.legend(fontsize=13)
  plt.xticks(fontsize=14)
  plt.yticks(fontsize=14)
  plt.show()


## Experiment methods

### No intervention

In [ ]:

def run_no_intervention_experiment(Profile: type[Patient], epoch_count: int = RunCount) -> ExperimentResult:
  sleep_hours = []
  positive_hours = []
  ratios = []
  notifications = []

  for _ in range(epoch_count):
    environment = Profile(
      behaviour_threshold=20,
      habituation=Habituation,
      time_preference_update_step=TimePreferenceUpdateStep
    )

    for _ in range(EightWeeksHours):
      observation, reward, done, _, info = environment.step(0)

    sleep_hours.append(environment.h_slept)
    positive_hours.append(environment.h_positive)
    ratios.append(environment.rr)
    notifications.append(environment.num_notified)

  return ExperimentResult(sleep_hours, positive_hours, ratios, notifications)


### Intervention

In [ ]:
from typing import Callable

def save_epoch_to_csv(environment: Patient, condition: str, name: str, epoch: int):
  df = pd.DataFrame()
  df['response_ratio'] = environment.rr
  df['activity performed'] = environment.num_performed
  df['notifications'] = environment.num_notified
  df['sleep'] = environment.h_slept
  df['positive'] = environment.h_positive
  df['non_stationary'] = environment.h_nonstationary
  df.to_csv(f"{ResultsDirectory}/patient1_{condition}_{name}_run{epoch}.csv")

Method = Callable[[Patient], Patient]
def run_intervention_experiment(Patient: type[Patient], method: Method, name: str, epoch_count=RunCount, condition=Condition):
  ratios = []
  notifications = []

  sleep_hours = []
  positive_mood_hours = []
  times_performed = 0

  for epoch in range(epoch_count):
    environment = method(
      Patient(
        behaviour_threshold=20,
        habituation=Habituation,
        time_preference_update_step=TimePreferenceUpdateStep
      )
    )

    if any(environment.num_performed): times_performed += 1

    save_epoch_to_csv(environment, condition, name, epoch)

    ratios.append(environment.rr)
    notifications.append(environment.num_notified)
    sleep_hours.append(environment.h_slept)
    positive_mood_hours.append(environment.h_positive)

  print("These figures include runs in which no prompt resulted in the activity being performed.")
  print(f"{times_performed} out of {epoch_count} resulted in activity being performed.")

  return ExperimentResult(sleep_hours, positive_mood_hours, ratios, notifications)


### Heuristic

Notify every hour except in night and when the patient is alseep

In [ ]:
def always_notify(environment: Patient, intervention_length: int = EightWeeksHours) -> Patient:
  action = 0

  for _ in range(intervention_length):
    observation, _, _, _, _ = environment.step(action)
    # 9-th is the time of the day
    # 3 is the night
    if observation[9] == 3 or observation[3] == 1:
      action = 0
    else:
      action = 1

  return environment

### Random 
Randomly sample during the day skip the nights

In [ ]:
def random_notification(environment: Patient, intervention_length: int = EightWeeksHours) -> Patient:
  action = 0
  for i in range(intervention_length):

    observation, _, _, _, _ = environment.step(action)
    # 9-th is the time of the day
    if observation[9] == 3:
      action = 0
    else:
      action = environment.action_space.sample()

  return environment

### Supervised Learning

#### Static model

In [ ]:
def supervised_after_three_weeks(environment: Patient) -> Patient:
  environment = always_notify(environment, intervention_length=ThreeWeeksHours)
  classifier = RandomForestClassifier(class_weight='balanced')
  classifier.fit(environment.observation_list, environment.activity_performed)

  remaining_time = EightWeeksHours - ThreeWeeksHours
  observation = environment._get_current_state()

  for _ in range(remaining_time):
    # applying supervised model
    action = classifier.predict(np.array([observation]))[0]
    observation, _, _, _, _ = environment.step(action)

  return environment

### Adaptive model 

In [ ]:
def supervised_adaptive_after_three_weeks(environment: Patient) -> Patient:
  environment = always_notify(environment, intervention_length=ThreeWeeksHours)
  classifier = RandomForestClassifier(class_weight='balanced')
  classifier.fit(environment.observation_list, environment.activity_performed)

  remaining_time = EightWeeksHours - ThreeWeeksHours
  observation = environment._get_current_state()
  samples = len(environment.observation_list)

  for _ in range(remaining_time):
    # applying supervised model
    action = classifier.predict(np.array([observation]))[0]
    observation, _, _, _, _ = environment.step(action)

    if len(environment.observation_list) <= samples: continue
    # retrain when new samples are provided
    classifier = RandomForestClassifier(class_weight='balanced')
    classifier.fit(environment.observation_list, environment.activity_performed)
    samples = len(environment.observation_list)

  return environment

### Deep Q Network

In [ ]:
def dqn(environment: Patient) -> Patient:
  model = DQN("MlpPolicy", environment, verbose=0, learning_starts=UpdatedEveryDayHours)
  model.learn(total_timesteps=EightWeeksHours)
  return environment

### Proximal Policy Optimisation

In [ ]:
def ppo(environment: Patient) -> Patient:
  model = PPO("MlpPolicy", environment, verbose=0, n_steps=UpdatedEveryDayHours, batch_size=UpdatedEveryDayHours)
  model.learn(total_timesteps=EightWeeksHours)
  return environment

### Advantage Actor Critic

In [ ]:
def a2c(environment: Patient) -> Patient:
  model = A2C("MlpPolicy", environment, verbose=0, n_steps=UpdatedEveryDayHours)
  model.learn(total_timesteps=EightWeeksHours)
  return environment

### Comparison between prompt learning strategies

In [ ]:
def run_experiment_for_profile(Profile: type[Patient], label: str):
  result = run_no_intervention_experiment(Profile)
  run_intervention_experiment(Profile, always_notify, f'h-{label}')
  run_intervention_experiment(Profile, random_notification, f'random-{label}')
  run_intervention_experiment(Profile, supervised_adaptive_after_three_weeks, f'static_sup3-{label}')
  run_intervention_experiment(Profile, supervised_adaptive_after_three_weeks, f'adaptive_sup3-{label}')
  run_intervention_experiment(Profile, dqn, f'dqn-{label}')
  run_intervention_experiment(Profile, ppo, f'ppo-{label}')
  run_intervention_experiment(Profile, a2c, f'a2c-{label}')

  always_notify_result = read_ratios_and_notifications(f'h-{label}')
  random_result = read_ratios_and_notifications(f'random-{label}')
  static_sup3_result = read_ratios_and_notifications(f'static_sup3-{label}')
  adaptive_sup3_result = read_ratios_and_notifications(f'adaptive_sup3-{label}')
  dqn_result = read_ratios_and_notifications(f'dqn-{label}')
  ppo_result = read_ratios_and_notifications(f'ppo-{label}')
  a2c_result = read_ratios_and_notifications(f'a2c-{label}')

  plot_experiment_result_hours(result)
  plot_experiment_results_ratio(random_result, {
    'Heuristic': always_notify_result,
    'Random': random_result,
    'RF Static': static_sup3_result,
    'RF Adaptive': adaptive_sup3_result,
    'DQN': dqn_result,
    'PPO': ppo_result,
    'A2C': a2c_result
  })

for (label, Profile) in profiles.items():
  print(f"Running experiment for profile: '{label}'")
  run_experiment_for_profile(Profile, label)
  print()
  